<a href="https://colab.research.google.com/github/cbonnin88/GreenBox/blob/main/Predictive_LTV_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# **Preparing the Data**

In [4]:
print('Preparing Data for ML')
df_users = pl.read_csv('greenbox_users.csv').with_columns(pl.col('signup_date').str.strptime(pl.Datetime,'%Y-%m-%d'))
df_events = pl.read_csv('greenbox_events.csv').with_columns(pl.col('timestamp').str.strptime(pl.Datetime,'%Y-%m-%d %H:%M:%S'))

data = df_events.join(df_users, on='user_id', how='left')
data = data.with_columns(((pl.col('timestamp') - pl.col('signup_date')).dt.total_days()).alias('days_since_signup'))
first_week_data = data.filter(pl.col('days_since_signup') <= 7)

Preparing Data for ML


In [26]:
# Features
features = (
    first_week_data.group_by('user_id')
    .agg([
        pl.col('event_name').filter(pl.col('event_name') == 'view_landing_page').len().alias('views_7d'),
        pl.col('event_name').filter(pl.col('event_name') == 'add_to_cart').len().alias('carts_7d'),
        pl.col('revenue').sum().alias('first_week_revenue')
    ])
)

# Target
target = data.group_by('user_id').agg(pl.col('revenue').sum().alias('actual_ltv'))

# Merge & Fill Nulls
ml_dataset = (
    df_users.join(features, on='user_id',how='left')
    .join(target, on='user_id',how='left')
    .fill_null(0)
)

# **Training the Model**

In [27]:
df_ml = ml_dataset.to_pandas()
df_ml = pd.get_dummies(df_ml, columns=['acquisition_channel','device'], drop_first=True)

X = df_ml.drop(columns=['user_id','signup_date','ab_test_group','age_groups','actual_ltv'])
y = df_ml['actual_ltv']

In [28]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = LinearRegression()
model.fit(X_train,y_train)
predictions = model.predict(X_test)

# **Visualize with Plotly**

In [29]:
plot_df = pd.DataFrame({'Actual LTV':y_test, 'Predicted LTV':predictions})

In [30]:
fig_ltv = px.scatter(
    plot_df,
    x='Actual LTV',
    y='Predicted LTV',
    title='Prediction Accuracy: Acutal vs Predicted TV',
    opacity=0.6,
    trendline='ols',
    trendline_color_override='red'
)

fig_ltv.add_shape(
    type='line',
    line=dict(dash='dash',color='grey'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)

fig_ltv.show()

# **Stats**

In [31]:
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)
print(f'RMSE: €{rmse:.2f} | R-Squared: {r2:.3f}')

RMSE: €11.67 | R-Squared: 0.914


# **K-MEANS LOGIC**

In [32]:
cluster_feature = df_ml[['views_7d','carts_7d','actual_ltv']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_feature)

In [33]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_ml['cluster'] = kmeans.fit_predict(X_scaled)

In [34]:
# Converting cluster to string so that Plotly treats it as a Category and not a number
df_ml['cluster'] = df_ml['cluster'].astype(str)

# **Visualize with Plotly**

In [39]:
fig_kmeans = px.scatter(
    df_ml,
    x='views_7d',
    y='actual_ltv',
    color='cluster',
    title='User Personas: Engagment (Views) vs Monetary Value (LTV)',
    hover_data = ['carts_7d'], # Show cart add when hovering
    color_discrete_sequence = px.colors.qualitative.Vivid,
    size='actual_ltv', # Make high value users bigger dots
    size_max=15,
    labels = {'actual_ltv':'Actual LTV','views_7d':'7 Day Views'}
)

fig_kmeans.show()

In [41]:
print('--- Cluster Averages ---')
display(df_ml.groupby('cluster')[['views_7d','carts_7d','actual_ltv']].mean())

--- Cluster Averages ---


,views_7d,carts_7d,actual_ltv
cluster,,,
0,1.0,1.0,144.204443
1,1.0,0.0,0.000000
2,1.0,1.0,0.095165
